In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
%pwd

'e:\\Chicken-Disease-Classification-Project'

In [2]:
# here we are creating a custom function return type using entity
from dataclasses import dataclass
from pathlib import Path

# data ingestion related config or entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [1]:
# importing paths from constants folder
from cnnClassifier.constants import * 
# importing some common general purpose functions
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        # read_yaml returns a ConfigBox datatype output
        # refer to config.yaml to see what content is stored in the configbox 
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # since config has ConfigBox datatype we can directly access its key like config.key
        create_directories([self.config.artifacts_root])

    # DataIngestionConfig class has been declared already
    # i can call this function to get all configurations related to our data
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion  

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

### Updating our Components

In [ ]:
import os
# to download dataset
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    # config parameter is of the data type DataIngestionConfig
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-12 18:18:41,597: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-12 18:18:41,621: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-12 18:18:41,623: INFO: common: created directory at: artifacts]
[2025-09-12 18:18:41,625: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-12 18:18:44,762: INFO: 1218392874: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11621297
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c59eb086f697a4734c2bda47f04f88206df50dfc5d0dcc683113e225dfde848"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EB33:5ECC7:EB602:211BC2:68C416A6
Accept-Ranges: bytes
Date: Fri, 12 Sep 2025 12:48:42 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210084-BOM
X-Ca